***
# TEST
***

In [ ]:
import os
import json
import pandas as pd
import pvdeg
from pytest import approx
from pvdeg import TEST_DATA_DIR

# Load weather data
WEATHER = pd.read_csv(
    os.path.join(TEST_DATA_DIR, "weather_day_pytest.csv"), index_col=0, parse_dates=True
)
with open(os.path.join(TEST_DATA_DIR, "meta.json"), "r") as file:
    META = json.load(file)

# Load expected results
rh_expected = pd.read_csv(
    os.path.join(TEST_DATA_DIR, "input_day_pytest.csv"), index_col=0, parse_dates=True
)
rh_cols = [col for col in rh_expected.columns if "RH" in col]
rh_expected = rh_expected[rh_cols]


def test_module():
    """
    test pvdeg.humidity.calc_rel_humidity

    Requires:
    ---------
    weather dataframe and meta dictionary
    """
    result = pvdeg.humidity.module(WEATHER, META)
    pd.testing.assert_frame_equal(result, rh_expected, check_dtype=False)


def test_psat():
    """
    test pvdeg.humidity.psat

    Requires:
    ---------
    weahter dataframe and meta dictionary
    """
    psat_avg = pvdeg.humidity.psat(temp=WEATHER["temp_air"])[1]
    assert psat_avg == approx(0.47607, abs=5e-5)

In [ ]:
test_psat()

In [ ]:
import pvdeg
import pickle
import pandas as pd
import numpy as np
import xarray as xr
import os
from pvdeg import TEST_DATA_DIR

GEO_META = pd.read_csv(os.path.join(TEST_DATA_DIR, "summit-meta.csv"), index_col=0)
with open(os.path.join(TEST_DATA_DIR, "summit-weather.pkl"), "rb") as f:
    GEO_WEATHER = pickle.load(f)

autotemplate

In [ ]:
autotemplate_result = pvdeg.geospatial.auto_template(
    func=pvdeg.humidity.module, ds_gids=GEO_WEATHER
).compute()

In [ ]:
humidity_template = xr.open_dataset(
    os.path.join(TEST_DATA_DIR, "humidity_template.nc")
).compute()

In [ ]:
def compare_datasets(ds1: xr.Dataset, ds2: xr.Dataset, atol=1e-10) -> bool:
    """Compare loaded datasets with "empty-like" values"""

    if ds1.dims != ds2.dims:
        return False

    if set(ds1.coords.keys()) != set(ds2.coords.keys()):
        return False

    for coord in ds1.coords:
        if ds1.coords[coord].dtype.kind in {"i", "f"}:
            # Use np.allclose for numeric coordinates
            if not np.allclose(ds1.coords[coord], ds2.coords[coord], atol=atol):
                return False
        elif ds1.coords[coord].dtype.kind == "M":  # datetime64 type
            # Use array equality for datetime coordinates
            if not np.array_equal(ds1.coords[coord], ds2.coords[coord]):
                return False
        else:
            if not np.array_equal(ds1.coords[coord], ds2.coords[coord]):
                return False

    if set(ds1.data_vars.keys()) != set(ds2.data_vars.keys()):
        return False

    for var in ds1.data_vars:
        if not np.allclose(ds1[var], ds2[var], atol=atol):
            return False

    for dim in ds1.dims:
        if not ds1.indexes[dim].equals(ds2.indexes[dim]):
            return False

    return True

In [ ]:
assert pvdeg.utilities.compare_datasets(autotemplate_result, humidity_template)

output template

In [ ]:
shapes = {
    "RH_surface_outside": ("gid", "time"),
    "RH_front_encap": ("gid", "time"),
    "RH_back_encap": ("gid", "time"),
    "RH_backsheet": ("gid", "time"),
}

manual_template = pvdeg.geospatial.output_template(
    shapes=shapes, ds_gids=GEO_WEATHER
).compute()

In [ ]:
pvdeg.utilities.compare_datasets(manual_template, humidity_template)

In [ ]:
# test template

shapes = {"testA": ("gid",), "testB": ("gid", "time")}

template = pvdeg.geospatial.output_template(
    shapes=shapes,
    ds_gids=GEO_WEATHER,
)

In [ ]:
template.to_netcdf(os.path.join(TEST_DATA_DIR, "mismatch-template.nc"))

In [ ]:
import pvdeg
from pvdeg import TEST_DATA_DIR
import pickle
import pandas as pd
import numpy as np
import xarray as xr
import os

GEO_META = pd.read_csv(os.path.join(TEST_DATA_DIR, "summit-meta.csv"), index_col=0)

with open(os.path.join(TEST_DATA_DIR, "summit-weather.pkl"), "rb") as f:
    GEO_WEATHER = pickle.load(f).compute().load()

HUMIDITY_TEMPLATE = xr.open_dataset(
    os.path.join(TEST_DATA_DIR, "humidity_template.nc"), engine="h5netcdf"
).compute()

In [ ]:
GEO_WEATHER

In [ ]:
GEO_WEATHER.chunks

In [ ]:
HUMIDITY_TEMPLATE.chunks

In [ ]:
shapes = {
    "RH_surface_outside": ("gid", "time"),
    "RH_front_encap": ("gid", "time"),
    "RH_back_encap": ("gid", "time"),
    "RH_backsheet": ("gid", "time"),
}

# falsely assigning chunks here
manual_template = pvdeg.geospatial.output_template(shapes=shapes, ds_gids=GEO_WEATHER)

assert pvdeg.utilities.compare_templates(manual_template, HUMIDITY_TEMPLATE)
for k, v in manual_template.chunks.items():
    if len(v) != 1:
        raise ValueError(f"""
                          Need one chunk per axis for an unchunked input
                          dimension {k} has {len(v)} chunks.
                          """)

In [ ]:
chunked_weather = GEO_WEATHER.chunk({"gid": 3})

In [ ]:
HUMIDITY_TEMPLATE.chunk({"gid": 3})

In [ ]:
chunked_template

In [ ]:
pvdeg.utilities.compare_templates(chunked_template, HUMIDITY_TEMPLATE.chunk({"gid": 3}))

In [ ]:
chunked_template = pvdeg.geospatial.auto_template(
    ds_gids=chunked_weather, func=pvdeg.humidity.module
)

geo_res = pvdeg.geospatial.analysis(
    weather_ds=chunked_weather,
    meta_df=GEO_META,
    func=pvdeg.humidity.module,
    template=chunked_template,
)

In [ ]:
geo_res = pvdeg.geospatial.analysis(
    chunked_weather,
    meta_df=GEO_META,
    func=pvdeg.humidity.module,
)

In [ ]:
res_ds = pvdeg.geospatial.analysis(
    weather_ds=GEO_WEATHER,
    meta_df=GEO_META,
    func=pvdeg.standards.standoff,
)

data_var = res_ds["x"]

# Stack the latitude and longitude coordinates into a single dimension
# convert to dataframe, this can be done with xr.dataset.to_dataframe as well
stacked = data_var.stack(z=("latitude", "longitude"))
latitudes = stacked["latitude"].values
longitudes = stacked["longitude"].values
data_values = stacked.values
combined_array = np.column_stack((latitudes, longitudes, data_values))

res = pd.DataFrame(combined_array).dropna()
ans = pd.read_csv(os.path.join(TEST_DATA_DIR, "summit-standoff-res.csv"), index_col=0)
res.columns = ans.columns

# pd.testing.assert_frame_equal(res, ans, check_dtype=False, check_names=False)

In [ ]:
res_ds.chunks

### Diffusion TESTing

In [1]:
import os
import pandas as pd
import numpy as np
import pytest
import pvdeg
from pvdeg import TEST_DATA_DIR
import json

In [2]:
WEATHER = pd.read_csv(
    os.path.join(TEST_DATA_DIR, "weather_day_pytest.csv"), index_col=0, parse_dates=True
)
with open(os.path.join(TEST_DATA_DIR, "meta.json"), "r") as file:
    META = json.load(file)

In [3]:
WEATHER

,temp_air,wind_speed,dhi,ghi,dni,relative_humidity
2021-01-01 00:00:00+00:00,-2.3,1.1,0,0,0,56.22
2021-01-01 00:10:00+00:00,-2.4,1.1,0,0,0,56.64
2021-01-01 00:20:00+00:00,-2.5,1.1,0,0,0,57.06
2021-01-01 00:30:00+00:00,-2.5,1.1,0,0,0,57.06
2021-01-01 00:40:00+00:00,-2.6,1.1,0,0,0,55.94
...,...,...,...,...,...,...
2021-01-01 23:10:00+00:00,-0.9,1.0,27,74,512,63.01
2021-01-01 23:20:00+00:00,-0.9,1.0,22,50,430,63.01
2021-01-01 23:30:00+00:00,-0.9,1.0,16,29,329,63.01
2021-01-01 23:40:00+00:00,-0.9,1.0,0,0,0,63.01


In [4]:
META

{'latitude': 39.7400016784668,
 'longitude': -105.16999816894531,
 'altitude': 1782,
 'tz': -7.0,
 'country': 'United States',
 'state': 'Colorado',
 'county': 'Jefferson',
 'wind_height': 2}

In [5]:
temperature = pvdeg.temperature.temperature(
    weather_df=WEATHER,
    meta=META,
    cell_or_mod="module", 
    temp_model="sapm",
    conf="open_rack_glass_polymer",
)

temperature = pd.DataFrame(temperature, columns = ['module_temperature'])
temperature['time'] = list(range(len(temperature)))

The array tilt angle was not provided, therefore the latitude tilt of 39.7 was used.
The array azimuth was not provided, therefore an azimuth of 180.0 was used.


In [6]:
temperature

,module_temperature,time
2021-01-01 00:00:00+00:00,-2.300000,0
2021-01-01 00:10:00+00:00,-2.400000,1
2021-01-01 00:20:00+00:00,-2.500000,2
2021-01-01 00:30:00+00:00,-2.500000,3
2021-01-01 00:40:00+00:00,-2.600000,4
...,...,...
2021-01-01 23:10:00+00:00,5.430119,139
2021-01-01 23:20:00+00:00,4.014083,140
2021-01-01 23:30:00+00:00,2.541752,141
2021-01-01 23:40:00+00:00,-0.900000,142


In [7]:
pressure = 0.2109 * (1 - 0.0065 * META['altitude'] / 288.15) ** 5.25588

oxygen_profile = pvdeg.diffusion.esdiffusion(
    temperature=temperature, 
    edge_seal='OX005', 
    encapsulant='OX003', 
    edge_seal_width=1.5, 
    encapsulant_width=10, 
    seal_nodes=20, 
    encapsulant_nodes=50, 
    press=pressure, 
    repeat=2
)

The edge seal is Helioseal_101_dry .
The encapsulant is EVA .


In [8]:
oxygen_profile

,0.000000,0.076923,0.153846,0.230769,0.307692,0.384615,0.461538,0.538462,0.615385,0.692308,...,9.681818,9.883838,10.085859,10.287879,10.489899,10.691919,10.893939,11.095960,11.297980,11.500000
0.0,0.000043,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1.0,0.000043,1.818119e-07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
2.0,0.000043,3.609618e-07,7.663460e-10,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
3.0,0.000043,5.387096e-07,2.281357e-09,3.230186e-12,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
4.0,0.000043,7.135628e-07,4.516240e-09,1.274835e-11,1.351509e-14,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
282.0,0.000041,2.498806e-05,1.416238e-05,6.900690e-06,2.898526e-06,0.000001,3.441013e-07,9.917788e-08,2.564298e-08,5.988366e-09,...,2.024059e-49,2.028521e-50,1.982055e-51,1.889086e-52,1.757121e-53,1.595790e-54,1.415725e-55,1.227532e-56,1.047591e-57,1.725855e-58
283.0,0.000042,2.502505e-05,1.418642e-05,6.922675e-06,2.913139e-06,0.000001,3.472982e-07,1.003339e-07,2.600639e-08,6.089116e-09,...,2.536217e-49,2.558377e-50,2.516060e-51,2.413649e-52,2.259629e-53,2.065468e-54,1.844261e-55,1.609419e-56,1.382442e-57,2.291883e-58
284.0,0.000042,2.506016e-05,1.420814e-05,6.942450e-06,2.926307e-06,0.000001,3.501923e-07,1.013829e-07,2.633703e-08,6.181021e-09,...,3.130883e-49,3.177629e-50,3.144255e-51,3.034781e-52,2.858541e-53,2.628911e-54,2.361702e-55,2.073534e-56,1.792069e-57,2.988721e-58
285.0,0.000043,2.508902e-05,1.422514e-05,6.957856e-06,2.936581e-06,0.000001,3.524600e-07,1.022067e-07,2.659725e-08,6.253527e-09,...,3.750303e-49,3.826644e-50,3.806710e-51,3.693835e-52,3.497940e-53,3.234153e-54,2.920949e-55,2.578227e-56,2.240279e-57,3.755784e-58


In [10]:
oxygen_profile.to_csv("1d-oxygen-profile.csv")

In [28]:
res = pd.read_csv(os.path.join(TEST_DATA_DIR, "1d-oxygen-profile.csv"), index_col=0, dtype="float64")

In [40]:
from copy import copy

col_list = copy(res.columns).values
col_list[21] = "1.5"

res.columns = col_list.astype(float)

res.columns

Index([                0.0, 0.07692307692307693, 0.15384615384615385,
       0.23076923076923078,  0.3076923076923077, 0.38461538461538464,
       0.46153846153846156,  0.5384615384615385,  0.6153846153846154,
        0.6923076923076923,  0.7692307692307693,  0.8461538461538463,
        0.9230769230769231,                 1.0,   1.076923076923077,
         1.153846153846154,  1.2307692307692308,  1.3076923076923077,
        1.3846153846153846,  1.4615384615384617,                 1.5,
                       1.5,   1.601010101010101,   1.803030303030303,
         2.005050505050505,   2.207070707070707,   2.409090909090909,
         2.611111111111111,   2.813131313131313,   3.015151515151515,
         3.217171717171717,   3.419191919191919,   3.621212121212121,
        3.8232323232323235,   4.025252525252525,  4.2272727272727275,
         4.429292929292929,  4.6313131313131315,   4.833333333333333,
        5.0353535353535355,   5.237373737373737,  5.4393939393939394,
         5.641414141

In [41]:
pd.testing.assert_frame_equal(
    oxygen_profile, res, 
    check_dtype=False, 
    check_column_type=False, 
)